# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
J. Olofsson  ->  J. Olofsson  |  ['J. Olofsson']
C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 52 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.05494


extracting tarball to tmp_2308.05494...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2308.05494/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'AUTHOR.tex' from 'tmp_2308.05494/AUTHOR.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2308.05735...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


✘ → 0:header
  ↳ 48924:end


/tmp/ipykernel_2154/3009462391.py:49: LatexWarning: 2308.05735 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.05613-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.05613) | **An inner warp discovered in the disk around HD 110058 using VLT/SPHERE  and HST/STIS**  |
|| S. Stasevic, et al. -- incl., <mark>J. Olofsson</mark>, <mark>C. Desgrange</mark>, <mark>E. Matthews</mark> |
|*Appeared on*| *2023-08-11*|
|*Comments*| *17 pages, 15 figures, 3 tables; accepted for publication in A&A*|
|**Abstract**| An edge-on debris disk was detected in 2015 around the young, nearby A0V star HD 110058. The disk showed features resembling those seen in the disk of beta Pictoris that could indicate the presence of a perturbing planetary-mass companion in the system. We investigated new and archival scattered light images of the disk in order to characterise its morphology and spectrum. In particular, we analysed the disk's warp to constrain the properties of possible planetary perturbers. Our work uses data from two VLT/SPHERE observations and archival data from HST/STIS. We measured the morphology of the disk by analysing vertical profiles along the length of the disk to extract the centroid spine position and vertical height. We extracted the surface brightness and reflectance spectrum of the disk. We detect the disk between 20 au (with SPHERE) and 150 au (with STIS), at a position angle of 159.6$^\circ\pm$0.6$^\circ$. Analysis of the spine shows an asymmetry between the two sides of the disk, with a 3.4$^\circ\pm$0.9$^\circ$ warp between ~20 au and 60 au. The disk is marginally vertically resolved in scattered light, with a vertical aspect ratio of 9.3$\pm$0.7% at 45 au. The extracted reflectance spectrum is featureless, flat between 0.95 micron and 1.1 micron, and red from 1.1 micron to 1.65 micron. The outer parts of the disk are also asymmetric with a tilt between the two sides compatible with a disk made of forward-scattering particles and seen not perfectly edge-on, suggesting an inclination of <84$^\circ$. The presence of an undetected planetary-mass companion on an inclined orbit with respect to the disk could explain the warp. The misalignment of the inner parts of the disk with respect to the outer disk suggests a warp that has not yet propagated to the outer parts of the disk, favouring the scenario of an inner perturber as the origin of the warp. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.05494-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.05494) | **ALMA Survey of Orion Planck Galactic Cold Clumps (ALMASOP): The  Warm-Envelope Origin of Hot Corinos**  |
|| S.-Y. Hsu, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-08-11*|
|*Comments*| *28 pages, 11 figures*|
|**Abstract**| Hot corinos are of great interest due to their richness in interstellar complex organic molecules (COMs) and the consequent potential prebiotic connection to solar-like planetary systems. Recent surveys have reported an increasing number of hot corino detections in Class 0/I protostars; however, the relationships between their physical properties and the hot-corino signatures remain elusive. In this study, our objective is to establish a general picture of the detectability of the hot corinos by identifying the origin of the hot-corino signatures in the sample of young stellar objects (YSOs) obtained from the Atacama Large Millimeter/submillimeter Array Survey of Orion Planck Galactic Cold Clumps (ALMASOP) project. We apply spectral energy distribution (SED) modeling to our sample and identify the physical parameters of the modeled YSOs directly, linking the detection of hot-corino signatures to the envelope properties of the YSOs. Imaging simulations of the methanol emission further support this scenario. We, therefore, posit that the observed COM emission originates from the warm inner envelopes of the sample YSOs, based on both the warm region size and the envelope density profile. The former is governed by the source luminosity and is additionally affected by the disk and cavity properties, while the latter is related to the evolutionary stages. This scenario provides a framework for detecting hot-corino signatures toward luminous Class 0 YSOs, with fewer detections observed toward similarly luminous Class I sources. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.05735-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.05735) | **A supermassive black hole in the early universe growing in the shadows**  |
|| L. J. Furtak, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-08-11*|
|*Comments*| *Submitted. Comments welcome!*|
|**Abstract**| Early JWST observations have uncovered a new, substantial population of red sources that might represent a previously overlooked phase of actively growing supermassive black holes (Kocevski et al. 2023, Matthee et al. 2023, Labbe et al. 2023). One of the most intriguing examples is an extremely red, point-like object that was found to be triply-imaged by the strong lensing galaxy cluster Abell 2744 (Furtak et al. 2023), allowing an unprecedented detailed look into this enigmatic population. Here we present deep spectroscopic JWST/NIRSpec observations of this object, Abell2744-QSO1. The spectroscopy confirms that the three images are of the same object, and that it is a highly reddened ($A_V\sim3$) broad emission-line Active Galactic Nucleus (AGN) at a redshift of $z_{\mathrm{spec}}=7.0451\pm0.0005$. From the width of H$\beta$ ($\mathrm{FWHM}=2800\pm250\,\frac{\mathrm{km}}{\mathrm{s}}$) we derive a black hole mass of $M_{\mathrm{BH}}=3_{-1}^{+2}\times10^7\,\mathrm{M}_{\odot}$. We infer a very high ratio of black hole to galaxy mass of at least 3% and possibly as high as 100%, an order of magnitude or more than is seen in local galaxies. The lack of strong metal lines in the spectrum together with the high bolometric luminosity ($L_{\mathrm{bol}}=(1.1\pm0.3)\times10^{45}\,\frac{\mathrm{erg}}{\mathrm{s}}$) suggest that we are seeing the black hole in a phase of rapid growth, accreting at 30% of the Eddington limit. Based on early JWST imaging studies we estimate that such heavily reddened, low-mass black holes can be $\sim100$ times more common than UV-selected ones at this epoch. The rapid growth and high black hole to galaxy mass ratio of A2744-QSO1 suggests that it may represent the missing link between black hole seeds (Inayoshi et al. 2020; Greene et al. 2020; Volonteri 2021) and the first luminous quasars (Fan et al. 2023). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.05613.md
    + _build/html/tmp_2308.05613/./HD-110058_Signal_Jy_arcsec-2_spine_fit_overlay.png
    + _build/html/tmp_2308.05613/./broad_band_overlay_STIS_pa_pc5_PCA_temp-mean.png
    + _build/html/tmp_2308.05613/./2015-04-12_ird_H23_RDI_temp-mean_pc_125.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bpic}[1]{\upbeta Pictoris}$
$\newcommand{\HD}[1]{HD 110058}$
$\newcommand{\micron}[1]{ \upmum}$
$\newcommand{\rev}[1]{#1}$</div>



<div id="title">

# An inner warp discovered in the disk around $\HD$  using VLT/SPHERE$\thanks{Based on observations made with ESO telescopes at the Paranal Observatory under programmes 095.C-0389(A) and ID 095.C-0607(A)}$ and HST/STIS$\thanks{Based on observations made with the STIS instrument aboard HST under program GO-15218 supported by NASA}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.05613-b31b1b.svg)](https://arxiv.org/abs/2308.05613)<mark>Appeared on: 2023-08-11</mark> -  _17 pages, 15 figures, 3 tables; accepted for publication in A&A_

</div>
<div id="authors">

S. Stasevic, et al. -- incl., <mark>J. Olofsson</mark>, <mark>C. Desgrange</mark>, <mark>E. Matthews</mark>

</div>
<div id="abstract">

**Abstract:** An edge-on debris disk was detected in 2015 around the young, nearby A0V star $\HD$ . The disk showed features resembling those seen in the disk of $\bpic$ that could indicate the presence of a perturbing planetary-mass companion in the system. We investigated new and archival scattered light images of the disk in order to characterise its morphology and spectrum. In particular, we analysed the disk's warp to constrain the properties of possible planetary perturbers. Using data from two VLT/SPHERE observations taken with the Integral Field Spectrograph (IFS) and near InfraRed Dual-band Imager and Spectrograph (IRDIS), we obtained high-contrast images of the edge-on disk. Additionally, we used archival data from HST/STIS with a poorer inner-working angle but a higher sensitivity to detect the outer parts of the disk. We measured the morphology of the disk by analysing vertical profiles along the length of the disk to extract the centroid spine position and vertical height. We extracted the surface brightness and reflectance spectrum of the disk. We detect the disk between 20 au (with SPHERE) and 150 au (with STIS), at a position angle of $159.6^\circ \pm 0.6^\circ$ . Analysis of the spine shows an asymmetry between the two sides of the disk, with a $3.4^\circ \pm 0.9^\circ$ warp between $\sim$ 20 au and 60 au. The disk is marginally vertically resolved in scattered light, with a vertical aspect ratio of $9.3 \pm 0.7\%$ at 45 au. The extracted reflectance spectrum is featureless, flat between 0.95 $\micron$ and 1.1 $\micron$ , and red from 1.1 $\micron$ to 1.65 $\micron$ . The outer parts of the disk are also asymmetric with a tilt between the two sides compatible with a disk made of forward-scattering particles and seen not perfectly edge-on, suggesting an inclination of $<84^\circ$ . The presence of an undetected planetary-mass companion on an inclined orbit with respect to the disk could explain the warp. The misalignment of the inner parts of the disk with respect to the outer disk suggests a warp that has not yet propagated to the outer parts of the disk, favouring the scenario of an inner perturber as the origin of the warp.

</div>

<div id="div_fig1">

<img src="tmp_2308.05613/./HD-110058_Signal_Jy_arcsec-2_spine_fit_overlay.png" alt="Fig2" width="100%"/>

**Figure 2. -** STIS observation of $\HD$ , the image rotated to have the disk horizontal, with the position of the brightest pixels (spine) overlaid in orange. The disk PA is $159.6^\circ$. (*fig:STIS_spine*)

</div>
<div id="div_fig2">

<img src="tmp_2308.05613/./broad_band_overlay_STIS_pa_pc5_PCA_temp-mean.png" alt="Fig3" width="100%"/>

**Figure 3. -** ADI-PCA reductions of HD110058 IFS data using 5 PCs for the Y, J, and H band, and combined YJH data, with centroid spine position overlaid (orange). The spine measured on the STIS data in the FOV is overlaid in white. The H band image contains only the 2015-04-03 observation, while the rest combine both epochs. Images are rotated by $70.4^\circ$ clockwise. (*fig:Spine*)

</div>
<div id="div_fig3">

<img src="tmp_2308.05613/./2015-04-12_ird_H23_RDI_temp-mean_pc_125.png" alt="Fig14" width="100%"/>

**Figure 14. -** RDI-PCA reduction of 2015-04-12 IRDIS data using 125 PCs for the H2 (left), H3 (centre), and combined H23 (right) channels. (*fig:RDI_img*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.05613"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

246  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

1  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
